# 8th Day of Python 4 DIP

> Contents:
> 1. Filters and local Filtering
> 2. Padding and Noises

Execute the cells to gain outputs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage as SK
import time
from Tools import *
%matplotlib inline

#### Mean Filter
$$Mean = \begin{bmatrix} 1/9 & 1/9 & 1/9\\1/9 & 1/9 & 1/9\\1/9 & 1/9 & 1/9\\\end{bmatrix}_{3\times 3}$$

In [ ]:
Camera = SK.io.imread("./Figures/cameraman.jpg",as_gray=True)*255
Padded_Camera = Zero_Padding(Camera,10,10)
Mean_Filter = np.full((3,3),1/9)

In [ ]:
ShowTime(Padded_Camera,With_Hist=True,bins=256,colorbar=True,Im_title="CameraMan")

## Convolution
Recall that the convolution of an image $f:\mathbb{R}^2\rightarrow \mathbb{R}$ and a kernel $h:\mathbb{R}^2\rightarrow\mathbb{R}$ is defined as follows:
$$(f*h)[m,n]=\sum_{i=-\infty}^\infty\sum_{j=-\infty}^\infty f[i,j]\cdot h[m-i,n-j]$$

Or equivalently,
\begin{align*}
(f*h)[m,n] &= \sum_{i=-\infty}^\infty\sum_{j=-\infty}^\infty h[i,j]\cdot f[m-i,n-j]\\
&= (h*f)[m,n]
\end{align*}



In [ ]:
def Convolution2D(Image,Kernel):
    """
    Simple convolution of Image with kernel.
    """
    M,N = Image.shape
    K1,K2 = Kernel.shape
    result = np.zeros_like(Image)
    Kernel = np.fliplr(np.flipud(Kernel))

    for y in range(M):
        for x in range(N):
            for i in range(K1):
                for j in range(K2):
                    if y+i-K1//2>=0 and y+i-K1//2 < M and x+j-K2//2>=0 and x+j-K2//2 < N:
                        result[y,x] += Kernel[i,j] * Image[y+i-K1//2,x+j-K2//2]

    return result 

In [ ]:
ShowTimeMul((Camera,Convolution2D(Camera,Mean_Filter)),figsize=(10,10),With_Hist=True,bins=256,Im_title=["Original","Mean Filtered"],His_Title=["Histogram"]*2)

#### Faster Convolution

In [ ]:
def FastConv2D(Image,Kernel):
    """
    Faster Convolution of 2D image with kernel.
    """
    M,N = Image.shape
    K1,K2 = Kernel.shape
    result = np.zeros_like(Image)
    Kernel = np.fliplr(np.flipud(Kernel))
    Padded_Image = Zero_Padding(Image,K1//2,K2//2)

    for y in range(M):
        for x in range(N):
            result[y,x] = np.sum(Padded_Image[y:y+K1, x:x+K2] * Kernel)
    
    return result

In [ ]:
%timeit Convolution2D(Camera,Mean_Filter)
%timeit FastConv2D(Camera,Mean_Filter)

#### Weighted Smoothing Filters

$$Weighted = \begin{bmatrix} 1/16 & 2/16 & 1/16\\2/16 & 4/16 & 2/16\\1/16 & 2/16 & 1/16\\\end{bmatrix}_{3\times 3}$$

In [ ]:
Weighted_Filter = 1/16 * np.array([[1,2,1],[2,4,2],[1,2,1]])

In [ ]:
ShowTimeMul((FastConv2D(Camera,Mean_Filter),FastConv2D(Camera,Weighted_Filter)),
            bins=256,With_Hist=True,figsize=(10,10),Im_title=["Mean Filter","Weighted average filter"],His_Title=["Histogram"]*2)

#### Blurring with Gaussian kernel

In [ ]:
Gaussian = np.array([
    [1,4,6,4,1],
    [4,16,24,16,4],
    [6,24,36,24,6],
    [4,16,24,16,4],
    [1,4,6,4,1]
])

ShowTimeMul((Camera,FastConv2D(Camera,Gaussian)/255),With_Hist=True,bins=256,Im_title=["Original","Convolved"],His_Title=["Histogram"]*2)

### Separablity of Filters

In [ ]:
a = 1/3*np.ones((3,1)).T
b = a.copy().T
print(f"The \n{a*b}\n equals to\n {Mean_Filter}.")

In [ ]:
%timeit Convolution2D(Convolution2D(Camera,a),b)
%timeit Convolution2D(Camera,Mean_Filter)

In [ ]:
g1 = np.array([[1,4,6,4,1]]).T
g2 = g1.copy().T
G = g1*g2
%timeit Convolution2D(Convolution2D(Camera,g1),g2)
%timeit Convolution2D(Camera,G)

In [ ]:
plt.imshow(SK.morphology.star(5))

#### Local Filtering with scikit-image

In [ ]:
## Mean Filter
%time Camera_Mean_SK = SK.filters.rank.mean(Camera/255,1/9*SK.morphology.square(3))
ShowTime(Camera_Mean_SK,With_Hist=True)

In [ ]:
## Median Filter
%time Camera_Med_SK = SK.filters.median(Camera,1/9*SK.morphology.square(9))
ShowTime(Camera_Med_SK,With_Hist=True,bins=256)

#### Padding with numpy
>>1. zero padding

In [ ]:
Chessboard = np.zeros((8,8))
Chessboard[0::2,1::2] = 1.0
Chessboard[1::2,0::2] = 1.0
Chess_Zero = np.pad(Chessboard,pad_width=3)
ShowTimeMul((Chessboard,Chess_Zero),With_Hist=False,Im_title=["Original","Zero padded"],colorbar=False)

>>2. Constant padding

In [ ]:
Chess_Const = np.pad(Chessboard,pad_width=3,mode='constant',constant_values=(1,0))
ShowTimeMul((Chessboard,Chess_Const),With_Hist=False,Im_title=["Original","Constant padded"],colorbar=False)

### Other modes

In [ ]:
Chess_Edge = np.pad(Chessboard,pad_width=3,mode="edge")
Chess_Min = np.pad(Chessboard,pad_width=3,mode="minimum")
Chess_Med = np.pad(Chessboard,pad_width=3,mode="median")
Chess_Mean = np.pad(Chessboard,pad_width=3,mode="mean")
Chess_Ref = np.pad(Chessboard,pad_width=3,mode="reflect")
Chess_Sym = np.pad(Chessboard,pad_width=3,mode="symmetric")
Chess_Wrap = np.pad(Chessboard,pad_width=3,mode="wrap")
ShowTimeMul((Chessboard,Chess_Edge,Chess_Min,Chess_Med,Chess_Mean,Chess_Ref,Chess_Sym,Chess_Wrap)
            ,With_Hist=False,colorbar=False,Im_title=["Original","edge","minimum","mean","median","reflect","symmetric","wrap"])

### Noises
>> Salt and pepper

In [ ]:
### Creation of Noise
np.random.seed(42)
probability = 0.01
Noise = np.random.random(size=Camera.shape)
Salt = Noise > probability
Camera_Salt = np.where(Salt,Camera,255)
Pepper = Noise > 1-probability
Camera_Pepper = np.where(Pepper,0,Camera)
Camera_Noised = np.where(Pepper,0,Camera_Salt)
ShowTimeMul((Camera,Camera_Salt,Camera_Pepper,Camera_Noised),figsize=(10,10),bins=256,With_Hist=False,colorbar=False,
            Im_title=["Original","Salt","Pepper","Salt & Pepper"],
            His_Title=["Histogram"]*4)


#### Using Scikit image

In [ ]:
Camera_Noised_SK = SK.util.random_noise(Camera,mode='s&p',seed=42,salt_vs_pepper=0.01)
ShowTimeMul((Camera,Camera_Noised),With_Hist=True,bins=256,figsize=(10,10)
            ,Im_title=["Original","Salt & Pepper"],His_Title=["Histogram"]*2)

#### Exersice 03
Two distinct images below have same histogram.

![](./Figures/Q1.png "Images")

1. `Will the blurred version of those images have same properties? Explain.`

2. `Repair those images with histogram filtering methods (2 method arbitrary) and compare their results.`

3.`Filter those images with Gaussian filter with std values of `$\sigma=0.5$`,`$\sigma=1.0$` and ` $\sigma = 2.0$` and compare their results.` 


>> Tip:
$$Gaussian(x,\mu,\sigma) = \frac{1}{\sqrt{2\pi\sigma^2}}e^{-\frac{(x-\mu)^2}{2\sigma^2}}$$

> **Deadline: `17/05/2023 12pm`.**